In [1]:
from sqlalchemy import create_engine
import zipfile
import os
import pandas as pd

# 1. Read the data

In [2]:
# Path to the zip file
zip_path = "D:/DA_Project/Data/Blinkit/blinkit_db.zip"
extract_path = "D:/DA_Project/Data/Blinkit"

# Extract the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# List the files after extraction
os.listdir(extract_path)


['blinkit_customers.csv',
 'blinkit_customer_feedback.csv',
 'blinkit_db.zip',
 'blinkit_delivery_performance.csv',
 'blinkit_inventory.csv',
 'blinkit_inventoryNew.csv',
 'blinkit_marketing_performance.csv',
 'blinkit_orders.csv',
 'blinkit_order_items.csv',
 'blinkit_products.csv',
 'Category_Icons.xlsx',
 'Rating_Icon.xlsx']

In [ ]:
# Path to the inventory file
orders_path = os.path.join(extract_path, "blinkit_orders.csv")
order_items_path = os.path.join(extract_path, "blinkit_order_items.csv")
product_path = os.path.join(extract_path, "blinkit_products.csv")
inventory_path = os.path.join(extract_path, "blinkit_inventory.csv")

# Read the data
orders_df = pd.read_csv(orders_path)
order_items_df = pd.read_csv(order_items_path)
product_df = pd.read_csv(product_path)

# 2. Data preprocessing

In [4]:
# Copy the original table
orders_df_clean = orders_df.copy()

# Check for duplicate data
orders_df_clean[orders_df_clean.duplicated(subset=["order_id"], keep=False)]


# Convert order_date to datetime
orders_df_clean["order_date"] = pd.to_datetime(orders_df_clean["order_date"])

# Add a time analysis column
orders_df_clean["order_day"] = orders_df_clean["order_date"].dt.date
orders_df_clean["order_month"] = orders_df_clean["order_date"].dt.month
orders_df_clean["order_year"] = orders_df_clean["order_date"].dt.year
orders_df_clean["order_hour"] = orders_df_clean["order_date"].dt.hour


In [5]:
# Copy the original table
order_items_df_clean = order_items_df.copy()

# Check for duplicate data
order_items_df_clean[order_items_df_clean.duplicated(subset=["order_id", "product_id"], keep=False)]

# Calculate the total value of each row
order_items_df_clean["item_total"] = order_items_df_clean["quantity"] * order_items_df_clean["unit_price"]

# Rename columns
order_items_df_clean.columns = [
    "order_id", "product_id", "quantity", "unit_price", "item_total"
]

# 3. Upload data to the database

In [7]:

# Connection information
server = "COMPUTER"
database = "BlinkitDB"

# Connect to SQL Server using the pymssql driver
conn_str = (
    f"mssql+pyodbc://@{server}/{database}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
)
engine = create_engine(conn_str)

# Push each table's data into SQL Server
orders_df_clean.to_sql("orders", con=engine, index=False, if_exists="replace")
order_items_df_clean.to_sql("order_items", con=engine, index=False, if_exists="replace")
product_df.to_sql("products", con=engine, index=False, if_exists="replace")


59